In [ ]:
!pip install openai==1.2.4

In [ ]:
import openai
import json
import pandas as pd
import re

In [ ]:
DATASET_PATH = ""

In [ ]:
df = pd.read_csv(DATASET_PATH)

In [ ]:
def preprocess_gender(gender_num, language):
    if language=='en':
        if gender_num==1:
            return 'male'
        if gender_num==2:
            return 'female'
        raise Exception('unknown gender')
    if language=='ru':
        if gender_num==1:
            return 'муж'
        if gender_num==2:
            return 'жен'
        raise Exception('unknown gender')
    raise Exception('unknown language')

    
def postprocess_gender(gender_num, language):
    if language=='en':
        if gender_num=='male':
            return 1
        if gender_num=='female':
            return 2
        raise Exception('unknown gender')
    if language=='ru':
        if gender_num=='муж':
            return 1
        if gender_num=='жен':
            return 2
        raise Exception('unknown gender')
    raise Exception('unknown language')

In [ ]:
nums_to_russian = {"contact_type": {1: "Телефон",  2: "Email", 3: "Skype", 4: "Telegram", 5: "Github"},
                   "education_type": {1: "Начальное", 2: "Повышение квалификации", 3: "Сертификаты", 4: "Основное"},
                   "education_level": {1: "Среднее", 2: "Среднее специальное", 3: "Неоконченное высшее", 4: "Высшее", 5: "Бакалавр", 6: "Магистр", 7: "Кандидат наук", 8: "Доктор наук"},
                    "language_level": {1: "Начальный", 2: "Элементарный",3: "Средний",4: "Средне-продвинутый",5: "Продвинутый",6: "В совершенстве",7: "Родной"}}

russian_to_nums = {"contact_type": {value: key for key, value in nums_to_russian["contact_type"].items()},
                  "education_type": {value: key for key, value in nums_to_russian["education_type"].items()},
                  "education_level": {value: key for key, value in nums_to_russian["education_level"].items()},
                  "language_level": {value: key for key, value in nums_to_russian["language_level"].items()}}


In [ ]:
nums_to_english = {"contact_type": {1: "Phone",  2: "Email", 3: "Skype", 4: "Telegram", 5: "Github"},
                   "education_type": {1: "Elementary", 2: "Training", 3: "Sertificates", 4: "Basic"},
                   "education_level": {1: "Secondary", 2: "Secondary professional", 3: "Higher incomplete", 4: "Higher", 5: "Bachelor", 6: "Master", 7: "Phd", 8: "Doctor"},
                    "language_level": {1: "Beginning", 2: "Pre-intermediate",3: "Intermediate",4: "Upper-intermediate",5: "Advanced",6: "Proficient",7: "Native"}}

english_to_nums = {"contact_type": {value: key for key, value in nums_to_english["contact_type"].items()},
                  "education_type": {value: key for key, value in nums_to_english["education_type"].items()},
                  "education_level": {value: key for key, value in nums_to_english["education_level"].items()},
                  "language_level": {value: key for key, value in nums_to_english["language_level"].items()}}

In [ ]:
def postprocess_special_fields(d):
    language = d['resume']['language']
    if language=='en':
        values = english_to_nums
    elif language=='ru':
        values = russian_to_nums
    else:
        raise Exception('unknown language')
    for i in range(len(d["resume"]["contactItems"])):
        d["resume"]["contactItems"][i]["contact_type"] = values["contact_type"].get(d["resume"]["contactItems"][i]["contact_type"], '')
    for i in range(len(d["resume"]["educationItems"])):
        d["resume"]["educationItems"][i]["education_type"] = values["education_type"].get(d["resume"]["educationItems"][i]["education_type"], '')
        d["resume"]["educationItems"][i]["education_level"] = values["education_level"].get(d["resume"]["educationItems"][i]["education_level"], '')
    for i in range(len(d["resume"]["languageItems"])):
        d["resume"]["languageItems"][i]["language_level"] = values["language_level"].get(d["resume"]["languageItems"][i]["language_level"], '')
    return d

In [ ]:
def preprocess_special_fields(d):
    language = d['resume']['language']
    if language=='en':
        values = nums_to_english
    elif language=='ru':
        values = nums_to_russian
    else:
        raise Exception('unknown language')
    for i in range(len(d["resume"]["contactItems"])):
        d["resume"]["contactItems"][i]["contact_type"] = values["contact_type"].get(d["resume"]["contactItems"][i]["contact_type"], '')
    for i in range(len(d["resume"]["educationItems"])):
        d["resume"]["educationItems"][i]["education_type"] = values["education_type"].get(d["resume"]["educationItems"][i]["education_type"], '')
        d["resume"]["educationItems"][i]["education_level"] = values["education_level"].get(d["resume"]["educationItems"][i]["education_level"], '')
    for i in range(len(d["resume"]["languageItems"])):
        d["resume"]["languageItems"][i]["language_level"] = values["language_level"].get(d["resume"]["languageItems"][i]["language_level"], '')
    return d

In [ ]:
def check_all_fields(d):
    for field in ["resume_id","first_name","last_name","middle_name","birth_date","birth_date_year_only","country","city","about","key_skills","salary_expectations_amount","salary_expectations_currency","photo_path","gender","resume_name","source_link","contactItems"]: 
        d["resume"][field]
    for i in d["resume"]["contactItems"]:
        for field in ["resume_contact_item_id", "value", "comment", "contact_type"]:
            i[field]
    for i in d["resume"]["educationItems"]:
        for field in ["resume_education_item_id", "year", "organization", "faculty", "specialty", "result", "education_type", "education_level"]:
            i[field]
    for i in d["resume"]["experienceItems"]:
        for field in ["resume_experience_item_id","starts","ends","employer","city","url","position","description","order"]:
            i[field]
    for i in d["resume"]["languageItems"]:
        for field in ["resume_language_item_id", "language", "language_level"]:
            i[field]


In [ ]:
def process_ints(item):
    for i in range(len(item["resume"]["contactItems"])):
        item["resume"]['contactItems'][i]['contact_type'] = int(item["resume"]['contactItems'][i]['contact_type'])
    for i in range(len(item["resume"]['educationItems'])):
        item["resume"]['educationItems'][i]['year'] = int(item["resume"]['educationItems'][i]['year'])  if len(str(item["resume"]['educationItems'][i]["year"]))>0 else ''
        item["resume"]['educationItems'][i]['education_type'] = int(item["resume"]['educationItems'][i]['education_type']) if len(str(item["resume"]['educationItems'][i]['education_type']))>0 else ''
        item["resume"]['educationItems'][i]['education_level'] = int(item["resume"]['educationItems'][i]['education_level']) if len(str(item["resume"]['educationItems'][i]['education_level']))>0 else ''
    for i in range(len(item["resume"]['experienceItems'])):
        item["resume"]['experienceItems'][i]['starts'] = int(item["resume"]['experienceItems'][i]['starts'])
        item["resume"]['experienceItems'][i]['ends'] = int(item["resume"]['experienceItems'][i]['ends']) if len(str(item["resume"]['experienceItems'][i]['ends']))>0 else ''
        item["resume"]['experienceItems'][i]['order'] = int(item["resume"]['experienceItems'][i]['order'])
    for i in range(len(item["resume"]['languageItems'])):
        item["resume"]['languageItems'][i]['language_level'] = int(item["resume"]['languageItems'][i]['language_level']) if len(str(item["resume"]['languageItems'][i]['language_level']))>0 else ''
    return item

In [ ]:
def preprocess_json(d):
    d = json.loads(d, strict=False)
    check_all_fields(d)
    d['resume']['key_skills'] = d['resume']['key_skills'].split(', ')
    d['resume']['gender'] = preprocess_gender(int(d['resume']['gender']), d['resume']['language'])
    d = process_ints(d)
    d = preprocess_special_fields(d)
    return d

In [ ]:
json_list = []
for  index, row in df.iterrows():
    try:
        json_list.append(preprocess_json(row['json']))
    except Exception as e:
        json_list.append('ERROR')
        print(e)
df['json'] = json_list
df = df[df["json"]!="ERROR"].reset_index(drop=True)
df

In [ ]:
english_prompt = """
    Extract information about candidate from this raw CV text. Return your answer in json format like this:
    ```
    {
      "resume": {
          "resume_id": "",
          "first_name": "",
          "last_name": "",
          "middle_name": "",
          "birth_date": "",
          "birth_date_year_only": "",
          "country": "",
          "city": "",
          "about": "",
          "key_skills": "",
          "salary_expectations_amount": "",
          "salary_expectations_currency": "",
          "photo_path": "",
          "gender": "",
          "language": "",
          "resume_name": "",
          "source_link": "",
          "contactItems": [
            {
              "resume_contact_item_id": "",
              "value": "",
              "comment": "",
              "contact_type": ""
            }
          ],
          "educationItems": [
            {
              "resume_education_item_id": "",
              "year": "",
              "organization": "",
              "faculty": "",
              "specialty": "",
              "result": "",
              "education_type": "",
              "education_level": ""
            }
          ],
          "experienceItems": [
            {
              "resume_experience_item_id": "",
              "starts": "",
              "ends": "",
              "employer": "",
              "city": "",
              "url": "",
              "position": "",
              "description": "",
              "order": ""
            }
          ],
          "languageItems": [
            {
              "resume_language_item_id": "",
              "language": "",
              "language_level": ""
            }
          ]
        }
    }
    ```
    Do not create any extra fields. Answer in English. Some fields can only have a limited number of values:
    ```
    1.  contact_type: {Phone, Email, Skype, Telegram, Github}
    Often contacts are written in the beginning of a CV.

    2. education_type: {Elementary, Training, Sertificates, Basic}
    This is usually about schools and courses of a candidate.

    3. education_level: {Secondary, Secondary professional, Higher incomplete, Higher, Bachelor, Master, PhD, Doctor}
    This field is usually about schools and courses of a candidate.

    4. language_level: {Beginning, Pre-intermediate, Intermediate, Upper-intermediate, Advanced, Proficient, Native}
    Sometimes candidates specify language_level information, but not always, so field can be empty. 
    ```
    """

In [ ]:
russian_prompt = """
Извлеки информацию о кандидате из этого текста резюме. Верни ответ в json формате следующего вида:
```
{
  "resume": {
      "resume_id": "",
      "first_name": "",
      "last_name": "",
      "middle_name": "",
      "birth_date": "",
      "birth_date_year_only": "",
      "country": "",
      "city": "",
      "about": "",
      "key_skills": "",
      "salary_expectations_amount": "",
      "salary_expectations_currency": "",
      "photo_path": "",
      "gender": "",
      "language": "",
      "resume_name": "",
      "source_link": "",
      "contactItems": [
        {
          "resume_contact_item_id": "",
          "value": "",
          "comment": "",
          "contact_type": ""
        }
      ],
      "educationItems": [
        {
          "resume_education_item_id": "",
          "year": "",
          "organization": "",
          "faculty": "",
          "specialty": "",
          "result": "",
          "education_type": "",
          "education_level": ""
        }
      ],
      "experienceItems": [
        {
          "resume_experience_item_id": "",
          "starts": "",
          "ends": "",
          "employer": "",
          "city": "",
          "url": "",
          "position": "",
          "description": "",
          "order": ""
        }
      ],
      "languageItems": [
        {
          "resume_language_item_id": "",
          "language": "",
          "language_level": ""
        }
      ]
    }
}
```
Не придумывай лишних полей. Отвечай на русском. Некоторые поля могут иметь только определенные значения:
```
1. Типы контактов — contact_type: {Телефон, Email, Skype, Telegram, Github}
Часто контакты указаны в начале резюме.

2. Виды образования — education_type: {Начальное, Повышение квалификации, Сертификаты, Основное}
Это информация про образование, которое получил кандидат.

3. Уровень образования — education_level: {Среднее, Среднее специальное, Неоконченное высшее, Высшее, Бакалавр, Магистр, Кандидат наук, Доктор наук}
Это информация про образование, которое получил кандидат.

3. Уровень знания языка — language_level: {Начальный, Элементарный, Средний, Средне-продвинутый, Продвинутый, В совершенстве, Родной}
Иногда кандидаты указывают эту информацию, но не всегда, поэтому поле может быть пустым.
```
"""

In [ ]:
train_dataset = []
for index, row in df.iterrows():
    if row['language'] == 'en':
        d = {"messages": [{"role": "system", "content": f"{english_prompt}"}, 
              {"role": "user", "content": f"{row['text']}"},
              {"role": "assistant", "content": f"{row['json']}"}]}
    elif row['language'] == 'ru':
        d = {"messages": [{"role": "system", "content": f"{russian_prompt}"}, 
              {"role": "user", "content": f"{row['text']}"},
              {"role": "assistant", "content": f"{row['json']}"}]}
    train_dataset.append(d)

with open("mydata.json", "w") as outfile:
    for d in train_dataset:
        outfile.write(json.dumps(d,  ensure_ascii=False) + "\n")

In [ ]:
len(train_dataset)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="",
)

In [ ]:
client.files.create(
  file=open("mydata.json", "rb"),
  purpose="fine-tune"
)

In [ ]:
client.fine_tuning.jobs.create(
    training_file="", 
    model="gpt-3.5-turbo-0125",
    hyperparameters={
    "n_epochs": 2
  }
)